In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import glob
import numpy as np
import tqdm

In [2]:
def read_aws(stn_id):
    dtypes = {"hd":str, "stn_id":str, "dt_lt":str, "dt_utc":str, "gust":str, "q":str, "#":str}
    df=pd.read_csv(glob.glob("/g/data/eg3/ab4502/ExtremeWind/obs/aws/*_one_min_gust/HD01D_Data_*"+stn_id+"*.txt")[0],\
        names=["hd","stn_id","dt_lt","dt_utc","gust","q","#"],header=0,dtype=dtypes)
    df["dt_utc"] = pd.DatetimeIndex(pd.to_datetime(df["dt_utc"], format="%Y%m%d%H%M"))
    df = df.set_index("dt_utc")
    df["gust"] = pd.to_numeric(df["gust"], errors="coerce") 
    
    return df

def latlon_dist(lat, lon, lats, lons):

        #Calculate great circle distance (Harversine) between a lat lon point (lat, lon) and a list of lat lon
        # points (lats, lons)
                        
        R = 6373.0
                        
        lat1 = np.deg2rad(lat)
        lon1 = np.deg2rad(lon)
        lat2 = np.deg2rad(lats)
        lon2 = np.deg2rad(lons)
                
        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

        return (R * c)
    
def load_lightning(date, stn_no, state):

    try:
        lightning = pd.read_csv("/g/data/eg3/ab4502/ExtremeWind/ad_data/lightning_raw/A"+date+".loc",header=None,
                               names=["date","time","lat","lon","1","2"])
    except:
        lightning = pd.read_csv("/g/data/eg3/ab4502/ExtremeWind/ad_data/lightning_raw/AE"+date+".loc",header=None,
                               names=["date","time","lat","lon","1","2","3","4","5"])
    names = ["id", "stn_no", "district", "stn_name", "site_open", "site_close", "lat", "lon", "latlon_method", "state",\
                    "hgt_asl", "hgt_asl_baro", "wmo_idx", "y1", "y2", "comp%", "Y%", "N%", "W%", "S%", "I%", "#"]
    stn_df = pd.read_csv(glob.glob("/g/data/eg3/ab4502/ExtremeWind/obs/aws/"+state+"_one_min_gust/HD01D_StnDet_*.txt")[0],\
            names=names, header=0)
    lightning["l_dist"] = latlon_dist(stn_df[stn_df.stn_no==int(stn_no)].lat.values[0], stn_df[stn_df.stn_no==int(stn_no)].lon.values[0],
        lightning["lat"], lightning["lon"])
    lightning["latlon"] = lightning["lat"].astype("str") + " " + lightning["lon"].astype("str")
    lightning["latlon"] = lightning.latlon.where(lightning.l_dist <= 50, np.nan)

    lightning["datetime"] = pd.DatetimeIndex(lightning["date"] + " " + lightning["time"])
    lightning = lightning.set_index(lightning["datetime"])    
    return lightning[["lat","lon","l_dist","latlon"]].resample("1min").nunique()    

In [3]:
details_list = [\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2006,9,24,1,41)], "rid":"2", "title":"a) Melbourne:  UTC", "stn_id":"086282","state":"vic"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2009,11,20,3,6)], "rid":"2", "title":"b) Melbourne:  UTC", "stn_id":"086282","state":"vic"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2010,3,6,3,30)], "rid":"2", "title":"c) Melbourne:  UTC", "stn_id":"086282","state":"vic"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2010,6,17,0,25)], "rid":"2", "title":"d) Melbourne:  UTC", "stn_id":"086282","state":"vic"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2011,11,18,7,4)], "rid":"2", "title":"e) Melbourne:  UTC", "stn_id":"086282","state":"vic"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2011,12,25,6,45)], "rid":"2", "title":"f) Melbourne:  UTC", "stn_id":"086282" ,"state":"vic"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2012,2,26,9,11)], "rid":"2", "title":"g) Melbourne:  UTC", "stn_id":"086282","state":"vic"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2013,3,21,3,45)], "rid":"2", "title":"h) Melbourne: UTC" , "stn_id":"086282","state":"vic"},\
        {"lat":-37.6654, "lon":144.8322, "time" : [dt.datetime(2015,2,28,9,25)], "rid":"2", "title":"i) Melbourne:  UTC", "stn_id":"086282","state":"vic"},\
       \
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2010,8,2,15,43)], "rid":"71", "title":"a) Sydney:  UTC", "stn_id":"066037","state":"nsw"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2012,8,10,4,21)], "rid":"71", "title":"b) Sydney: UTC", "stn_id":"066037","state":"nsw"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2013,2,23,13,43)], "rid":"71", "title":"c) Sydney: UTC", "stn_id":"066037","state":"nsw"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2014,10,14,11,39)], "rid":"71", "title":"d) Sydney: UTC", "stn_id":"066037","state":"nsw"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2015,3,1,4,34)], "rid":"71", "title":"e) Sydney: UTC", "stn_id":"066037","state":"nsw"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2015,12,16,2,3)], "rid":"71", "title":"f) Sydney: UTC", "stn_id":"066037","state":"nsw"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2016,1,14,4,20)], "rid":"71", "title":"g) Sydney: UTC", "stn_id":"066037","state":"nsw"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2016,6,4,3,42)], "rid":"71", "title":"h) Sydney: UTC", "stn_id":"066037","state":"nsw"},\
        {"lat":-33.9465, "lon":151.1731, "time" : [dt.datetime(2017,4,9,8,3)], "rid":"71", "title":"i) Sydney: UTC", "stn_id":"066037","state":"nsw"},\
       \
        {"lat":-27.6297, "lon":152.7111, "time" : [dt.datetime(2010,12,16,3,25)], "rid":"66", "title":"a) Amberley: UTC", "stn_id":"040004","state":"qld"},\
        {"lat":-27.6297, "lon":152.7111, "time" : [dt.datetime(2011,1,18,5,5)], "rid":"66", "title":"b) Amberley: UTC", "stn_id":"040004","state":"qld"},\
        {"lat":-27.4034, "lon":151.7413, "time" : [dt.datetime(2011,10,7,19,22)], "rid":"50", "title":"c) Oakey: UTC", "stn_id":"041359","state":"qld"},\
        {"lat":-27.4034, "lon":151.7413, "time" : [dt.datetime(2013,10,18,4,4)], "rid":"50", "title":"d) Oakey: UTC", "stn_id":"041359","state":"qld"},\
        {"lat":-27.6297, "lon":152.7111, "time" : [dt.datetime(2013,11,23,9,45)], "rid":"66", "title":"e) Amberley: UTC", "stn_id":"040004","state":"qld"},\
        {"lat":-27.4034, "lon":151.7413, "time" : [dt.datetime(2014,1,23,4,52)], "rid":"50", "title":"f) Oakey UTC", "stn_id":"041359","state":"qld"},\
        {"lat":-27.4034, "lon":151.7413, "time" : [dt.datetime(2016,1,29,4,22)], "rid":"50", "title":"g) Oakey UTC", "stn_id":"041359","state":"qld"},\
        {"lat":-27.6297, "lon":152.7111, "time" : [dt.datetime(2016,12,18,4,14)], "rid":"66", "title":"h) Amberley: UTC", "stn_id":"040004","state":"qld"},\
        {"lat":-27.4034, "lon":151.7413, "time" : [dt.datetime(2018,2,13,7,5)], "rid":"50", "title":"i) Oakey: UTC", "stn_id":"041359","state":"qld"},\
       \
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2007,10,27,9,3)], "rid":"27", "title":"a) ", "stn_id":"016001","state":"sa"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2008,12,9,6,18)], "rid":"27", "title":"b) ", "stn_id":"016001","state":"sa"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2010,12,7,9,27)], "rid":"27", "title":"c) ", "stn_id":"016001","state":"sa"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2011,11,8,9,47)], "rid":"27", "title":"d) ", "stn_id":"016001","state":"sa"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2012,1,29,4,42)], "rid":"27", "title":"e) ", "stn_id":"016001","state":"sa"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2012,11,30,8,33)], "rid":"27", "title":"f) ", "stn_id":"016001","state":"sa"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2014,10,31,7,52)], "rid":"27", "title":"g) ", "stn_id":"016001","state":"sa"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2015,12,7,11,20)], "rid":"27", "title":"h) ", "stn_id":"016001","state":"sa"},\
        {"lat":-31.1558, "lon":136.8054, "time" : [dt.datetime(2017,12,18,11,30)], "rid":"27", "title":"i) ", "stn_id":"016001","state":"sa"},\
         ]

In [4]:
time_ls = []
rid = []
wg = []
rb = []
ra = []
rab4 = []
rab1 = []
pr = []
lightning=[]

for detail in tqdm.tqdm(details_list):
    event_time = detail["time"][0]
    for f in glob.glob("/g/data/eg3/ab4502/ExtremeWind/obs/aws/one_min_case_data_2/HD01D_Data_"+detail["stn_id"]+"_*.txt"):
        df = pd.read_csv(f,
                   names=["hd","stn_no","dt_lt","Time (UTC)","p","p_q","p_p","t","t1","dp","dp_q",\
                          "rh","rh_q","ws","ws_q","min_ws","min_ws_q","wd","wd_q","std_wd","std_wd_q",\
                          "wg","wg_q","mslp","mslp_q","pres","pres_q","qnh","qnh_q","#"], header=0, index_col="Time (UTC)", parse_dates=True)
        df["wg"] = pd.to_numeric(df["wg"],errors="coerce")
        df["p"] = pd.to_numeric(df["p"],errors="coerce")
        if np.min(np.abs(df.index - event_time)).seconds < 3600:
                time = df.index[np.argmax(df.wg)]
                times = [time + dt.timedelta(seconds=-2*60*60), 
                         time + dt.timedelta(seconds=-30*60), 
                         time + dt.timedelta(seconds=30*60), 
                         time + dt.timedelta(seconds=2*60*60)]
                df = df.set_index(pd.DatetimeIndex(df.index))
                df = df.merge(load_lightning(detail["time"][0].strftime("%Y%m%d"),
                                             detail["stn_id"],detail["state"]), how="outer", right_index=True, left_index=True)
                wg_max = df.wg.max()

                time_ls.append(time)
                rid.append(detail["rid"])
                wg.append(wg_max)
                if df.loc[slice(times[0], time)].wg.isna().sum() >= 30:
                    rb.append(np.nan)
                else:
                    rb.append(wg_max / df.loc[slice(times[0], time)].wg.mean())
                if df.loc[slice(time, times[3])].wg.isna().sum() >= 30:
                    ra.append(np.nan)
                else:
                    ra.append(wg_max / df.loc[slice(time, times[3])].wg.mean())
                rab4.append(wg_max / df.loc[slice(times[0], times[3])].wg.mean())
                rab1.append(wg_max / df.loc[slice(times[1], times[2])].wg.mean()) 
                
                if (df.loc[slice(time, time+dt.timedelta(seconds=60*60))].p.sum() < 1) & \
                        df.loc[slice(time, time+dt.timedelta(seconds=60*60))].p.isna().sum() < 10:
                    pr.append(df.loc[slice(time, time+dt.timedelta(seconds=60*60))].p.sum())
                else:
                    pr.append(np.nan)
                
                lightning.append(df.loc[slice(time+dt.timedelta(seconds=-30*60), time+dt.timedelta(seconds=30*60))].latlon.sum())
                
                break

100%|██████████| 36/36 [06:28<00:00, 10.79s/it]


In [5]:
pd.DataFrame({"time":time_ls, "rid":rid, "wg":wg, "rb":rb, "ra":ra, "rab4":rab4, "rab1":rab1, "pr":pr, "lightning":lightning}).to_csv("/g/data/eg3/ab4502/figs/ExtremeWind/case_studies/one_min_obs_stats.csv")